<a href="https://colab.research.google.com/github/worldbank/dec-python-course/blob/main/1-foundations/4-api-and-dataviz/foundations-s4-api-bonus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

## Argument-based parameters

- The example of geoBoundaries we previously saw takes _URL-based_ API query parameters
- Many APIs take _argument-based_ query parameters
- This will be the case every time you have to use query parameters separated by an ampersand symbol (`&`) after a question mark (`?`) in the API endpoint
- Take this generic example:

`https://api.org/endpoint/?parameter1=value1&parameter2=value2&parameter3=value3`

- Theoretically, it's _possible_ to modify an API URL call using concatenated strings in Python to build argument-based queries

In [ ]:
endpoint = 'https://api.org/endpoint/'
p1 = 'parameter1'
v1 = 'value1'
url = endpoint+'?'+p1+'='+v1
print(url)

- Then we could use `requests` as usual to obtain a response from this API

```{python}
requests.get(url)
```

- However, the convention in Python is to **use the argument `params` of `requests.get()`** to pass argument-based query parameters

```{python}
parameters = {'parameter1': 'value1'}
requests.get(endpoint, params=parameters)
```

### Applied example: the WBG API

To illustrate an example of argument-based parameters, we'll use again the endpoint of the total population API to fetch country population data. Note that this time the parameters `date` and `year` are passed in the dictionary named `parameters` and not in the URL string as we did before.

In [ ]:
def fetch_population_by_year(year):
    
    endpoint = 'https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL'
    parameters = {'date': year, 'format':'json'}
    # note: the API documentation specifies that format=json
    # is a required parameter in order to return the results as JSON
    response = requests.get(endpoint, params=parameters)
    
    if response.status_code == 200:
        
        data = response.json()
        return data
    
    else:
        
        print('Request failed!')
        return None

In [ ]:
pop_2015 = fetch_population_by_year(2015)

In [ ]:
pop_2015

A few notes:
- Remember that this is the same data you obtain when accessing https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?date=2015&format=json on a web browser
- The first element of this list contains metadata about the data returned by the API

In [ ]:
pop_2015[0]

- Note the detail of the information in the keys `page`, `pages`, `per_page`, and `total` in the first element of `pop_2015`

In [ ]:
print('Total obs: {}'.format(pop_2015[0]['total']))
print('Obs per page: {}'.format(pop_2015[0]['per_page']))
print('Total pages: {}'.format(pop_2015[0]['pages']))
print('Current page: {}'.format(pop_2015[0]['page']))

- The result is only one page with 50 observations, out of a total of 266
- This means that results are incomplete! More API calls are needed to complete the total 266 observations
- A further inspection of the API documentation shows that the parameters `page=page_number` or `per_page=obs_per_page` can be used to retrieve complete results.

**Important:** When fetching data using APIs, always inspect the result to look for possible limitations in the results' default format. You might be inadvertently missing observations in your API calls!

## Bonus exercise

Modify the function `fetch_population_by_year()` below to retrieve the complete results for a given year (not only page 1) from the population endpoint.

Suggested steps:

1. Run `fetch_population_by_year()` for any year and inspect the resulting list. You will note that the first element of the list is a dictionary with a key `total` that indicates the total number of observations in the query result
1. In your function, save that number into a variable
1. Send a new API request adding the parameter `per_page` in the parameters dictionary. Its value should be the total number of observations
1. The JSON object in the response content will be a new list with the same format: the first element of the list will contain metadata about the API result and the data will be in the second element
1. Return the second element of your list

In [ ]:
def fetch_population_by_year(year):
    
    endpoint = 'https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL'
    parameters = {'date': year, 'format':'json'}
    response = requests.get(endpoint, params=parameters)
    
    if response.status_code == 200:
        
        data = response.json()
        
        # === ADD YOUR SOLUTION HERE ===
        # Note that this time we're not giving you the steps
        # to follow and variables to create. Figuring that
        # out is also part of the exercise :)
        
        return data
    
    else:
        
        print('Request failed!')
        return None

## Coding API clients - More takeaways

- Passing API call parameters as arguments is a more elegant way of coding API clients. Argument-based parameters also work better when introducing non-conventional characters as arguments. Think, for example: how would you include a space character in a URL? argument-based parameters handle those cases seamlessly
- When authentication is needed, API keys are usually passed as argument-based parameters. If the API has a dedicated client library, they will ask for the key after importing the library (as `user_agent` in `Nominatim()`)
- Many APIs will combine the use of URL-based and argument-based parameters to pass information in API queries. A good API client will take that into account to build the correct query URL and parameters argument. Take the following example:

In [ ]:
def fetch_population_by_year_country(year, country):
    
    endpoint = 'https://api.worldbank.org/v2/country/' + country + '/indicator/SP.POP.TOTL'
    parameters = {'date': year, 'format':'json'}
    # note: the API documentation specifies that format=json
    # is a required parameter in order to return the results as JSON
    response = requests.get(endpoint, params=parameters)
    
    if response.status_code == 200:
        
        data = response.json()
        return data
    
    else:
        
        print('Request failed!')
        return None

In [ ]:
brazil_data = fetch_population_by_year_country(2015, 'BRA')

In [ ]:
brazil_data